In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-26T09:34:28.427574-07:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.13.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.0-1017-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

/home/dgxuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%watermark --iversions

xgboost: 1.7.5
logging: 0.5.1.2
dask   : 2023.3.2
pandas : 1.5.3
numpy  : 1.23.5
optuna : 3.1.1



In [5]:
!nvidia-smi -L

GPU 0: NVIDIA H100 80GB HBM3 (UUID: GPU-5c583ee7-8fb1-b26a-4bea-16e45d984a32)
GPU 1: NVIDIA H100 80GB HBM3 (UUID: GPU-a6ab06f5-a6e2-18c1-9dec-0dcd29f44a46)
GPU 2: NVIDIA H100 80GB HBM3 (UUID: GPU-bb8d5098-3c56-c48d-a0a3-fcdfcec6d3f5)
GPU 3: NVIDIA H100 80GB HBM3 (UUID: GPU-cdbe686b-1611-999b-8e8d-a3c5f35b40c4)
GPU 4: NVIDIA H100 80GB HBM3 (UUID: GPU-0df1cef0-fc95-cc88-b5f4-239889b3acba)
GPU 5: NVIDIA H100 80GB HBM3 (UUID: GPU-9dca657c-dbe4-08b7-fe7c-5653f183f0b6)
GPU 6: NVIDIA H100 80GB HBM3 (UUID: GPU-33389782-e2ad-5022-997d-cf470313879c)
GPU 7: NVIDIA H100 80GB HBM3 (UUID: GPU-926eaa05-87fb-35e0-0a98-adb2ad41d0be)


In [6]:
cluster = LocalCUDACluster(n_workers=8)
client = Client(cluster)

2023-05-26 09:34:33,964 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-26 09:34:33,964 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-26 09:34:33,965 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-26 09:34:33,965 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-26 09:34:33,966 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-26 09:34:33,966 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-26 09:34:33,969 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-26 09:34:33,969 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-05-26 09:34:33,971 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-05-26 09:34:33,971 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-

In [7]:
!nvidia-smi

Fri May 26 09:34:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA H100 80G...  On   | 00000000:1B:00.0 Off |                    0 |
| N/A   33C    P0   110W / 700W |    525MiB / 81559MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA H100 80G...  On   | 00000000:43:00.0 Off |                    0 |
| N/A   

In [8]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_l.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_l.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [9]:
train = pd.read_csv('../input/train.csv.zip')

shift = 200

target0 = train['loss'].values
target = np.log(target0+shift)

In [10]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': 'reg:squarederror', 
        'tree_method':'gpu_hist',  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['rmse']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = np.exp(predictions) - shift
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    mae = mean_absolute_error(target0, train_oof)
    
    return mae

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_l_3.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_allstate_l_3.db", study_name="five_fold_optuna_xgb_l_3", direction='minimize')

In [12]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/tmp/ipykernel_2449241/4104002597.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:13: FutureWarnin

CPU times: user 28.2 s, sys: 20.7 s, total: 48.9 s
Wall time: 6min 19s


In [13]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1148.387211,0.006142,0.429734,rmse,0.017136,2.103571,0.007091,12,16,0.513130,COMPLETE
1,1,2236.530649,3.075554,0.764214,rmse,0.002917,2.182807,0.002024,23,115,0.520454,COMPLETE
2,2,1183.441715,0.001184,0.593583,rmse,0.049443,0.001806,0.028585,22,12,0.448310,COMPLETE


In [14]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l_3.csv', index=False)
df

/tmp/ipykernel_2449241/4104002597.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:13: FutureWarnin

CPU times: user 13.3 s, sys: 6.97 s, total: 20.3 s
Wall time: 3min


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1148.387211,0.006142,0.429734,rmse,0.017136,2.103571,0.007091,12,16,0.513130,COMPLETE
1,1,2236.530649,3.075554,0.764214,rmse,0.002917,2.182807,0.002024,23,115,0.520454,COMPLETE
2,2,1183.441715,0.001184,0.593583,rmse,0.049443,0.001806,0.028585,22,12,0.448310,COMPLETE
3,3,1152.306285,0.001045,0.636107,rmse,0.041021,0.283415,0.006932,24,262,0.472992,COMPLETE
4,4,1185.626653,0.003449,0.860554,rmse,8.841536,5.605923,0.012689,5,281,0.418027,COMPLETE
5,5,1169.135121,2.391024,0.362789,rmse,0.970304,0.001605,0.034259,3,137,0.489251,COMPLETE
6,6,1137.062159,0.020223,0.567169,rmse,0.200575,0.497313,0.013589,16,100,0.649279,COMPLETE
7,7,2806.705500,0.001266,0.778426,rmse,2.460758,0.180863,0.001396,21,296,0.876090,COMPLETE


In [15]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_l_3.csv', index=False)
df.head()

/tmp/ipykernel_2449241/4104002597.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma': trial.suggest_loguniform('gamma',1e-3,10.0),
/tmp/ipykernel_2449241/4104002597.py:13: FutureWarnin

CPU times: user 5min 19s, sys: 4min 12s, total: 9min 32s
Wall time: 1h 17min 2s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1148.387211,0.006142,0.429734,rmse,0.017136,2.103571,0.007091,12,16,0.513130,COMPLETE
1,1,2236.530649,3.075554,0.764214,rmse,0.002917,2.182807,0.002024,23,115,0.520454,COMPLETE
2,2,1183.441715,0.001184,0.593583,rmse,0.049443,0.001806,0.028585,22,12,0.448310,COMPLETE
3,3,1152.306285,0.001045,0.636107,rmse,0.041021,0.283415,0.006932,24,262,0.472992,COMPLETE
4,4,1185.626653,0.003449,0.860554,rmse,8.841536,5.605923,0.012689,5,281,0.418027,COMPLETE


In [16]:
df

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_gamma,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,1148.387211,0.006142,0.429734,rmse,0.017136,2.103571,0.007091,12,16,0.513130,COMPLETE
1,1,2236.530649,3.075554,0.764214,rmse,0.002917,2.182807,0.002024,23,115,0.520454,COMPLETE
2,2,1183.441715,0.001184,0.593583,rmse,0.049443,0.001806,0.028585,22,12,0.448310,COMPLETE
3,3,1152.306285,0.001045,0.636107,rmse,0.041021,0.283415,0.006932,24,262,0.472992,COMPLETE
4,4,1185.626653,0.003449,0.860554,rmse,8.841536,5.605923,0.012689,5,281,0.418027,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
103,103,1136.944736,0.043987,0.418437,rmse,0.020153,1.863738,0.016058,16,105,0.700389,COMPLETE
104,104,1136.809254,0.032591,0.479991,rmse,0.009172,1.703855,0.014628,18,118,0.728447,COMPLETE
105,105,1137.223147,0.033314,0.419830,rmse,0.009836,1.939132,0.015473,18,141,0.730939,COMPLETE
106,106,1137.300539,0.075294,0.395806,rmse,0.018631,1.705826,0.014189,20,129,0.748893,COMPLETE


In [17]:
df.value.min()

1136.426586258589